# Data Warehouse Utilization

In [1]:
%load_ext sql

In [2]:
%%sql
sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

## Which classroom is the most utilized, and what programs hold classes there? (Done)

In [3]:
%%sql
SELECT location, COUNT(course_id) as count
FROM LOCATIONS_DW 
    JOIN CLASS_FACTS_DW USING (location_id)
    JOIN COURSES_DW USING (course_id)
GROUP BY location 
ORDER BY count DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


location,count
DSB 112,171
BNW 341,160
DSB 109,158
NHS 301,158
CNS 101,157
DSB 106,157
CNS 108,154
BNW 137,152
CNS 106,149
DSB 108,148


In [4]:
%%sql
SELECT program_name AS Program, count(catalog_id) AS Count
FROM COURSES_DW 
    JOIN CLASS_FACTS_DW USING (course_id)
    JOIN PROGRAMS_DW USING (program_id)
    JOIN LOCATIONS_DW USING (location_id)
    WHERE location = 'DSB 112'
    GROUP BY program_name
    ORDER BY Count DESC; 

 * sqlite:///CourseDataWarehouse.db
Done.


Program,Count
Accounting,63
Marketing,38
Management,26
Finance,21
Operations Management,10
Information Systems,8
Taxation,5


## Which classes go over capacity the most frequently?

Problem- why are some professors teaching multiple sections of the same course? 6 sections of AC 0012 by same prof? check if accurate...

In [20]:
%%sql
SELECT DISTINCT Course_title, COUNT(Course_title) AS Count
FROM (SELECT DISTINCT COURSES_DW.CatalogYear, term, section, professor_id, COURSES_DW.Course_id, Catalog_id, Course_title, Timecodes, Cap, Actual, Remaining
FROM COURSES_DW    
    JOIN CLASS_FACTS_DW USING (course_id)
    JOIN TIMECODES_DW USING(Timecode_id)
    WHERE Remaining < 0 AND Cap != 0
    ORDER BY professor_id)
GROUP BY course_title
ORDER BY Count DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Course_title,Count
Introduction to Management Accounting,13
Operations Management,13
Principles of Marketing,13
Introduction to Information Systems,11
Marketing Research,10
Business Strategies in the Global Environment,8
Health Assessment Lab,7
Honors Seminar,7
Introduction to Finance,7
Introduction to Management,7


In [9]:
--SUBQUERY
%%sql
SELECT DISTINCT COURSES_DW.CatalogYear, term, section, professor_id, COURSES_DW.Course_id, Catalog_id, Course_title, Timecodes, Cap, Actual, Remaining
FROM COURSES_DW    
    JOIN CLASS_FACTS_DW USING (course_id)
    JOIN TIMECODES_DW USING(Timecode_id)
    WHERE Remaining < 0 AND Cap != 0 
    --GROUP BY course_title
    ORDER BY professor_id
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


CatalogYear,Term,Section,Professor_id,Course_id,Catalog_id,Course_title,Timecodes,Cap,Actual,Remaining
2017_2018,Spring2018,01,2,135,AC 0012,Introduction to Management Accounting,['MR 1100am-1215pm 01/16-05/01 DSB 108'],25,27,-2
2017_2018,Spring2018,02,2,135,AC 0012,Introduction to Management Accounting,['MR 1230pm-0145pm 01/16-05/01 DSB 108'],25,29,-4
2017_2018,Fall2017,01,3,161,AC 0330,Auditing,['MR 0200pm-0315pm 09/05-12/11 DSB 112'],26,27,-1
2017_2018,Fall2017,02,3,161,AC 0330,Auditing,['MR 0330pm-0445pm 09/05-12/11 DSB 112'],26,27,-1
2017_2018,Spring2018,07,3,135,AC 0012,Introduction to Management Accounting,['MR 0930am-1045am 01/16-05/01 DSB 111'],25,28,-3
2017_2018,Spring2018,08,3,135,AC 0012,Introduction to Management Accounting,['MR 1100am-1215pm 01/16-05/01 DSB 111'],25,28,-3
2017_2018,Spring2018,01,3,315,AC 0555,Issues in Internal Audit,['MR 0630pm-0930pm 03/19-05/11 NHS 404'],15,17,-2
2018_2019,Spring2019,01,3,2536,AC 0560,Audit Issues in a Global Environment,['MR 0150pm-0450pm 01/28-03/15 NHS 103'],15,24,-9
2018_2019,Spring2019,02,3,2536,AC 0560,Audit Issues in a Global Environment,"['M 0150pm-0450pm 03/25-05/10 NHS 103', 'S 0900am-0530pm 03/30-03/30 DSB 111', 'S 0900am-0530pm 04/13-04/13 DSB 111', 'S 0900am-0530pm 04/27-04/27 DSB 111']",15,17,-2
2017_2018,Fall2017,01,4,187,AC 0343,Federal Income Taxation I,['W 0630pm-0900pm 09/05-12/11 DSB 110B'],24,26,-2


## Which professors have the most diverse courseload? (DONE)

In [21]:
%%sql
SELECT professor_id, Name, COUNT(DISTINCT Course_id) AS CourseCount
FROM PROFESSORS_DW
    JOIN CLASS_FACTS_DW USING (professor_id)
    JOIN COURSES_DW USING (course_id)
    GROUP BY professor_id, Name
    ORDER BY CourseCount DESC
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Professor_id,Name,CourseCount
161,Jeffrey N. Denenberg,18
110,Michael P. Pagano,17
65,Aaron R. Van Dyke,16
111,Qin Zhang,15
511,Amalia I. Rusu,15
576,Shannon P. Gerry,14
67,Diane J. Brousseau,13
84,John R. Miecznikowski,13
93,Amanda S. Harper-Leatherman,13
117,Virginia A. Kelly,13


In [22]:
%%sql
SELECT DISTINCT term, catalog_id, course_title
FROM COURSES_DW
    JOIN CLASS_FACTS_DW USING (course_id)
    WHERE professor_id = '161'; 

 * sqlite:///CourseDataWarehouse.db
Done.


Term,Catalog_id,Course_title
Fall2017,ECE 0461,Green Power Generation
Fall2017,EE 0213L,Electric Circuits Lab
Fall2017,EE 0231,Introduction to Electronics Circuits and Devices
Fall2017,EE 0231L,Electronics Circuits Lab
Fall2017,EE 0361,Green Power Generation
Fall2018,BEN 0331,Biomedical Signal Processing
Fall2018,CR 0331,Biomedical Signal Processing
Fall2018,EE 0213L,Electric Circuits Lab
Fall2018,EE 0231L,Electronics Circuits Lab
Spring2018,CR 0245L,Digital Design I Lab


## On which day are the most/least classes held? (DONE)

In [6]:
%%sql
SELECT DISTINCT Day, count(Day) as Count
FROM TIMECODES_DW
GROUP BY Day
ORDER BY Count DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


Day,Count
T,249
W,243
R,206
M,196
F,137
S,45
U,14


## During which time slot are the most classes held?

In [7]:
%%sql
SELECT COUNT(*)
FROM TIMECODES_DW
WHERE Start = '12:30:00';

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(*)
49


In [31]:
%%sql
SELECT DISTINCT Start,End,COUNT(Start),COUNT(End), Day,course_title
FROM TIMECODES_DW
    JOIN CLASS_FACTS_DW USING (timecode_id)
    JOIN COURSES_DW USING (course_id)
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Start,End,COUNT(Start),COUNT(End),Day,Course_title
11:00:00,12:15:00,10316,10316,M,Introduction to Financial Accounting


In [34]:
%%sql
SELECT DISTINCT Start,End, COUNT(Start) AS Popular_Start_time, COUNT(End) AS Popular_End_time
FROM TIMECODES_DW
    --JOIN CLASS_FACTS_DW USING (timecode_id)
    --JOIN COURSES_DW USING (course_id)
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Start,End,Popular_Start_time,Popular_End_time
07:00:00,10:00:00,1090,1090


In [ ]:
%%sql
SELECT DISTINCT Start, COUNT (Start)End, Day,course_title
FROM TIMECODES_DW
    JOIN CLASS_FACTS_DW USING (timecode_id)
    JOIN COURSES_DW USING (course_id)
LIMIT 10;

## Which professor has taught the most classes? students? (use actual, not cap)

Comment- double check if this is right. Might be over counting bc of same problem as first question

In [8]:
%%sql
SELECT Professor_id AS ID, program_name AS Program, Name, COUNT(Course_title) AS Courses_Taught
FROM PROFESSORS_DW    
    JOIN CLASS_FACTS_DW USING (professor_id)
    JOIN PROGRAMS_DW USING (program_id)
    JOIN COURSES_DW USING (course_id)
    GROUP BY ID
    ORDER BY Courses_Taught DESC
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ID,Program,Name,Courses_Taught
984,Accounting,Victor E. Hartman,72
374,Nursing,Mary M Murphy,67
9,Accounting,Milo W. Peck,63
457,Physics,Bidyut B. Das,57
842,Spanish,Lauren S. Gaskill,56
383,Nursing,Laura A. Conklin,54
379,Nursing,Kelly Ann Sullivan,51
14,Accounting,Patricia M. Poli,47
251,Sociology,Rose P. Rodrigues,46
398,Nursing,Martha Ann Boulton,46


In [9]:
%%sql
SELECT Professor_id AS ID, program_name AS Program, Name, SUM(actual) AS Students_Taught
FROM PROFESSORS_DW    
    JOIN CLASS_FACTS_DW USING (professor_id)
    JOIN PROGRAMS_DW USING (program_id)
    JOIN COURSES_DW USING (course_id)
    GROUP BY ID
    ORDER BY Students_Taught DESC
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ID,Program,Name,Students_Taught
374,Nursing,Mary M Murphy,3249
380,Nursing,Katherine Marie Saracino,1611
9,Accounting,Milo W. Peck,1598
51,Biology,Christine Earls,1319
52,Biology,Anthony J. DeCristofaro,1190
457,Physics,Bidyut B. Das,1121
307,Marketing,John E. Neal,1110
984,Accounting,Victor E. Hartman,1077
297,Management,Paul P. Daulerio,1058
496,Religious Studies,John E. Thiel,1008


## Which classes are attracting <10 students? (which classes are the least popular--- ratio?)

In [46]:
%%sql
-- Find classes where actual is <10 
-- Also get the count of each value
SELECT Course_title as Course, Catalog_id, Name, COURSES_DW.CatalogYear, Term, Actual, Cap, round((Actual*1.0/Cap*1.0),2) AS Ratio
    FROM COURSES_DW
    JOIN CLASS_FACTS_DW USING (course_id)
    JOIN PROFESSORS_DW USING (professor_id)
    JOIN TIMECODES_DW USING (timecode_id)
    JOIN PROGRAMS_DW USING (program_id)
    WHERE Actual < 10 AND Ratio <0.3
    GROUP BY Course_title
    ORDER BY Actual DESC

LIMIT 20;

 * sqlite:///CourseDataWarehouse.db
Done.


Course,Catalog_id,Name,CatalogYear,Term,Actual,Cap,Ratio
Precalculus,MA 0011,Scott Kaminski,2017_2018,Fall2017,9,32,0.28
Introduction to Probability and Statistics,MA 0017,Riaz A. Lalani,2017_2018,Fall2017,8,32,0.25
"Population: Birth, Death, and Migration",SO 0184,Mehmet Cansoy,2018_2019,Fall2018,8,28,0.29
20th Century United States,HI 0239,David W. McFadden,2017_2018,Summer2018,7,25,0.28
Abstract Algebra,MA 0436,Paul Baginski,2017_2018,Fall2017,7,25,0.28
Comprehensive Exam in Marriage and Family Therapy,FT 0099,Erica E. Hartwell,2018_2019,Fall2018,7,25,0.28
Politics of Humanitarian Action,PO 0129,Janie L. Leatherman,2017_2018,Fall2017,7,25,0.28
Popular Music Theory and Composition,MU 0155,Brian Q. Torff,2018_2019,Fall2018,7,25,0.28
Social Change in Developing Nations,SO 0191,Marcela Aliaga,2017_2018,Fall2017,7,24,0.29
Calculus II,MA 0172,Neha Hooda,2018_2019,Spring2019,6,25,0.24
